In [1]:
# Clean Script
from glob import glob
import tensorflow as tf
from tensorflow.keras.utils import load_img

# Collect paths to images based on label
nonCancerPaths = glob('imagedata/*/0/*.png')
cancerousPaths = glob('imagedata/*/1/*.png')

# Turn filepaths into image arrays to train a model
def paths_to_image(paths, label):
    images = []
    labels = []
    for path in paths:
        image = load_img(path)
        image = image.resize([50, 50])
        imgArray = tf.keras.utils.img_to_array(image)
        images.append(imgArray)
        labels.append(label)
    return (images, labels)


2023-03-19 11:59:30.481611: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from PIL import Image
import cv2
def rotate_image(paths, label, rotation):
    images = []
    labels = []
    for path in paths:
        image = load_img(path)
        im = Image.open(path)
        image = image.resize([50, 50])
        rotateimg = im.rotate(rotation)
        imgArray = tf.keras.utils.img_to_array(rotateimg)
        images.append(imgArray)
        labels.append(label)
    return (images, labels)



In [3]:
import random 

randomnCPaths = random.choices(nonCancerPaths, k=37000)

nonCancerImages = paths_to_image(nonCancerPaths, 0)
noncancerImages90 = rotate_image(randomnCPaths, 1,90)

cancerImages = paths_to_image(cancerousPaths, 1)
cancerImages90 = rotate_image(cancerousPaths, 1,90)
cancerImages180 = rotate_image(cancerousPaths, 1,180)

In [ ]:
print(np.array(nonCancerImages).shape

In [ ]:
from PIL import Image
Image.open('imagedata/10257/1/10257_idx5_x1801_y1551_class1.png')

In [ ]:
from PIL import Image
Image.open('imagedata/9036/0/9036_idx5_x2351_y1151_class0.png')

In [ ]:
img = load_img('data/train/cats/cat.0.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='preview', save_prefix='cat', save_format='jpeg'):
    i += 1
    if i > 20:
        break  

In [ ]:
# Train Test Split train = (images, labels)
from sklearn.model_selection import train_test_split
allImages = nonCancerImages[0] + cancerImages[0] + noncancerImages90[0] + cancerImages90[0] + cancerImages180[0]
allLabels = nonCancerImages[1] + cancerImages[1]+ noncancerImages90[1] + cancerImages90[1] + cancerImages180[1]

x_train, x_test, y_train, y_test = train_test_split(allImages, allLabels, test_size=.5, stratify= allLabels, random_state = 10)


In [ ]:
# define the model according to keras tutorial 
# https://www.tensorflow.org/tutorials/images/cnn#create_the_convolutional_base
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np


print(np.array(x_train).shape)
print(np.array(y_train).shape)

#Add data augmentation to increase the robustness of the model
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
])


model = models.Sequential()
model.add(data_augmentation)
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(50, 50, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(np.array(x_train), np.array(y_train), epochs=10, validation_data=(np.array(x_test), np.array(y_test)))

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(np.array(x_test),  np.array(y_test), verbose=2)